In [1]:
import os
os.environ["PYSPARK_PYTHON"] = os.path.join(os.environ["CONDA_PREFIX"], "python.exe")
os.environ["PYSPARK_DRIVER_PYTHON"] = os.path.join(os.environ["CONDA_PREFIX"], "python.exe")
print ("PYSPARK_PYTHON: ", os.environ["PYSPARK_PYTHON"])
print ("PYSPARK_DRIVER_PYTHON: ", os.environ["PYSPARK_DRIVER_PYTHON"])

PYSPARK_PYTHON:  C:\Users\lamma\anaconda3\envs\UIT_model\python.exe
PYSPARK_DRIVER_PYTHON:  C:\Users\lamma\anaconda3\envs\UIT_model\python.exe


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

kafka_topic_name = "songTopic"
kafka_bootstrap_servers = 'localhost:9092'

scala_version = '2.13'
spark_version = '3.5.5'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}','org.apache.kafka:kafka-clients:2.10.0']
spark = SparkSession.builder.master('local')\
.appName("kafka-example").config("spark.jars.packages", ",".join(packages))\
.config("spark.memory.offHeap.enabled","true")\
.config("spark.memory.offHeap.size","10g")\
.config("spark.driver.memory", "32g")\
.getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Construct a streaming DataFrame that reads from test-topic
songs_raw_stream = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers)\
    .option("subscribe", kafka_topic_name)\
    .load()

songs_df = songs_raw_stream.selectExpr("CAST(value AS STRING)")
songs_schema_string = "order_id INT,id STRING, name STRING,popularity INT, duration_ms DOUBLE, explicit INT, " \
                           + "artists STRING, id_artists STRING, release_date STRING, " \
                           + "danceability DOUBLE," \
                           + "energy DOUBLE, key INT, loudness DOUBLE, " \
                           + "mode INT," \
                           + "speechiness DOUBLE," \
                           + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, " \
                           + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"



songs_df = songs_df \
        .select(from_csv(col("value"), songs_schema_string) \
                .alias("song"), "value") \
        .select("song.*", "value") \

songs_agg_write_stream = songs_df \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("Song") \
        .start()

songs_agg_write_stream.awaitTermination(1)



False

In [3]:
import pandas as pd
from spotify_api import getSong
song_data = getSong.passs()
#song_data.rename(columns={'duration_s': 'duration_ms' }, inplace=True)
song_data = song_data.drop(['id','duration_s'], axis='columns')
rand_n = random. randint(1,len(song_data)-1)
add_df = song_data.head(rand_n)[-1:]

Index(['id', 'name', 'popularity', 'duration_s', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')


In [12]:
add_df
df_sp = spark.createDataFrame(add_df)

In [24]:
df = spark.sql("SELECT * FROM Song")

df = df.sort(df.release_date.desc())

df_stream = df

df = df.drop('order_id',
 'id',
 'explicit',
  'mode',
 'release_date',
 'id_artists',
 'time_signature',
 'duration_ms',
 'timestamp',
 'value')

In [25]:
for x in range (100):
    print (df.toPandas())

                                                  name  popularity  \
0    Folsom Prison Blues - Live at Folsom State Prison         NaN   
1          Cocaine Blues - Live at Folsom State Prison         NaN   
2                 Busted - Live at Folsom State Prison         NaN   
3    Dark as the Dungeon - Live at Folsom State Prison         NaN   
4    A Boy Named Sue - Live at San Quentin State Pr...         NaN   
..                                                 ...         ...   
290                                      Via del campo        51.0   
291                                     Both Sides Now        51.0   
292                                     Shifting Sands        51.0   
293                   I Heard It Through The Grapevine        51.0   
294                                 Le nozze di Figaro         NaN   

                                          artists  danceability  energy  key  \
0                                          162653           NaN     NaN  NaN   

In [26]:
df.show(truncate=False)

+--------------------------------------------------+----------+------------------------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+------+
|name                                              |popularity|artists                             |danceability|energy|key |loudness|speechiness|acousticness|instrumentalness|liveness|valence|tempo |
+--------------------------------------------------+----------+------------------------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+------+
|Folsom Prison Blues - Live at Folsom State Prison |NULL      |162653                              |NULL        |NULL  |NULL|0.524   |-15.269    |1.0         |0.0803          |0.0652  |0.00199|0.893 |
|Cocaine Blues - Live at Folsom State Prison       |NULL      |169293                              |NULL        |NULL  |NULL|0.571   |-12.317    |1.0         |0.103           |0.27    |0.0    |0.9

In [27]:
df_sp = df_sp.drop('order_id',
 'id',
 'explicit',
  'mode',
 'release_date',
 'id_artists',
 'time_signature',
 'duration_ms',
 'timestamp')

df_sp.show(truncate=False)

+---------+----------+---------------+------------+------+---+--------+-----------+------------+----------------+--------+-------+-------+
|name     |popularity|artists        |danceability|energy|key|loudness|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |
+---------+----------+---------------+------------+------+---+--------+-----------+------------+----------------+--------+-------+-------+
|Ave Maria|0         |['Dick Haymes']|0.227       |0.261 |5  |-12.343 |0.0382     |0.994       |0.247           |0.0977  |0.0539 |118.891|
+---------+----------+---------------+------------+------+---+--------+-----------+------------+----------------+--------+-------+-------+



In [28]:
df = df.union(df_sp)


In [29]:
df.show(truncate=False)

+--------------------------------------------------+----------+------------------------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+------+
|name                                              |popularity|artists                             |danceability|energy|key |loudness|speechiness|acousticness|instrumentalness|liveness|valence|tempo |
+--------------------------------------------------+----------+------------------------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+------+
|Folsom Prison Blues - Live at Folsom State Prison |NULL      |162653                              |NULL        |NULL  |NULL|0.524   |-15.269    |1.0         |0.0803          |0.0652  |0.00199|0.893 |
|Cocaine Blues - Live at Folsom State Prison       |NULL      |169293                              |NULL        |NULL  |NULL|0.571   |-12.317    |1.0         |0.103           |0.27    |0.0    |0.9

In [30]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=[
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo'], outputCol='features')
assembled_data=assembler.setHandleInvalid("skip").transform(df).persist()



In [31]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
df=data_scale.transform(assembled_data).persist()

In [32]:
df.show(truncate=False)

+------------------------------------------------+----------+--------------------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+-------+--------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                                            |popularity|artists                         |danceability|energy|key |loudness|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |features                                                      |standardized                                                                                                                                                                       |
+------------------------------------------------+----------+--------------------------------+------------+------+----+---

In [33]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo=KMeans(featuresCol='standardized', k=2)
    
KMeans_fit=KMeans_algo.fit(df)
    
output_df =KMeans_fit.transform(df)

In [35]:

import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

class SpotifyRecommender():
    def __init__(self, rec_data):
        self.rec_data_ = rec_data
    
    def spotify_recommendations(self, song_name, amount=1):
        distances = []
        song = self.rec_data_[(self.rec_data_.name.str.lower() == song_name.lower())].head(1).values[0]
        # get details of our fav song from name we pass as x earlier.
        res_data = self.rec_data_[self.rec_data_.name.str.lower() != song_name.lower()]
        #dropping the data with our fav song so that it doesnt affect our recommendation.
        for r_song in tqdm(res_data.values):
            # tqdm is just used for showing the bar of iteration through our streamed songs.
            dist = 0
            for col in np.arange(len(res_data.columns)):
                # (len(res_data.columns) gets us the number of columns -> 13 in our case.
                #indeces of non-numerical columns neednt be considered.
                if not col in [0,1,13]:
                    #calculating the manhettan distances for each numerical feature
                    # song -> from our fav dataset.
                    # r_song -> from streaming data.
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
            # distances are calculated and appended and added to a new column called distances in our dataset.
        res_data['distance'] = distances
        #sorting our data to be ascending by 'distance' feature
        res_data = res_data.sort_values('distance')
        # resulting dataset have the song similar to our fav song's numerical values and thus recommended.
        columns = ['name', 'artists', 'acousticness', 'liveness', 'instrumentalness', 'energy', 'danceability', 'valence']
        return res_data[columns][:amount]
    
    
datad = output_df.select('name',
 'artists',
 'danceability',
 'energy',
 'key',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'prediction')



datf = datad.toPandas()
datf.drop(datf[datf['artists'] == '0'].index, inplace = True)
datf.drop_duplicates(inplace=True)
datf.drop(datf[datf['danceability'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['liveness'] == 0.000].index, inplace = True)
datf.drop(datf[datf['instrumentalness'] == 0.000000].index, inplace = True)
datf.drop(datf[datf['energy'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['danceability'] == 0.000].index, inplace = True)
datf.drop(datf[datf['valence'] == 0.000].index, inplace = True)

y = datf
value_pred = datf.iloc[-1:]['prediction']
#datf = datf[datf['prediction'] == list(value_pred)[0]]

recommender = SpotifyRecommender(datf)
x = add_df['name'].tolist()[0]

rec_song = recommender.spotify_recommendations(x, 10)

v = add_df[['name', 'artists',  'acousticness', 'liveness', 'instrumentalness', 'energy', 
       'danceability', 'valence']]

rec_song = pd.concat([rec_song, v])
rec_song.to_csv('rec_song.csv')

100%|██████████| 203/203 [00:00<00:00, 50756.70it/s]


In [36]:
df_rec = spark.createDataFrame(rec_song)
df_rec.show()

+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|                name|             artists|acousticness|liveness|instrumentalness|energy|danceability|valence|
+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|           Evil Ways|         ['Santana']|        0.22|    0.14|            0.35| 0.528|       0.704|   0.73|
|  Time Of The Season|     ['The Zombies']|       0.127|   0.275|         3.83E-6| 0.571|       0.541|  0.666|
|(You're The) Devi...|   ['Elvis Presley']|       0.481|   0.242|         0.00141| 0.579|       0.483|  0.891|
|    Suspicious Minds|   ['Elvis Presley']|      0.0422|   0.411|         4.74E-6| 0.382|       0.487|  0.714|
|Glass Onion - Rem...|     ['The Beatles']|      0.0123|   0.108|         8.38E-5|  0.51|       0.523|  0.789|
| I Wanna Be Your Dog|     ['The Stooges']|      0.0229|   0.419|           0.155| 0.871|       0.527|  0.107|
|

### Verification for feature selection

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
import pandas as pd


df = df_stream
assembler = VectorAssembler(inputCols = pca_col, outputCol = 'features')
df1 = assembler.transform(df.na.drop("any"))

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df1)


scaledData = scaler.transform(df1)

a = scaledData.select('scaledFeatures')

pca = PCA(k=10,inputCol="scaledFeatures", outputCol="pcaFeatures").fit(scaledData)

df_pca = pca.transform(scaledData)

df3 = df_pca.select('pcaFeatures')

# Trains a k-means model.
kmeans = KMeans(k=4)
model = kmeans.fit(df_pca)

# Make predictions
predictions = model.transform(df_pca)



from  pyspark.ml.linalg import Vectors
temp = predictions.select("pcaFeatures")
temp = temp.rdd.map(lambda x: [float(y) for y in x['pcaFeatures']]).toDF(['PC1'])




pandas_df = temp.toPandas()
predictions_df = predictions.toPandas()

pred = predictions_df['prediction']

final_df = pd.concat([pandas_df,pred],axis =1)

df_pca_res = spark.createDataFrame(final_df)

df_pca_res.show(5)

+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+----------+
|                PC1|                 _2|                  _3|                 _4|                 _5|                  _6|                 _7|                  _8|                  _9|                 _10|prediction|
+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+----------+
| 0.7746811601486698| -2.232961385510888|  0.7057761424582696|0.08010424507519297|-3.1548626924927174|  2.0596441046915337|0.24698658995414946|  0.4790522980488063|0.003166937177649...|  0.2755632052807099|         2|
|-1.5720568137345565|-0.4423527752405789| -0.9794354110927322|0.16541739894466267| 0.5541714379755487|0.037495492292427435|-0.79

In [ ]:
pca.explainedVariance

DenseVector([0.259, 0.1337, 0.1135, 0.1028, 0.0968, 0.0904, 0.0774, 0.0554, 0.0517, 0.0193])